<a href="https://colab.research.google.com/github/karansubedi/GroceryBasket/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy

In [ ]:
!python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
doc = nlp("Donald Trump was President of USA")

In [ ]:
doc.ents

from spacy import displacy
displacy.render(doc,style="ent",jupyter=True)

In [ ]:
import pandas as pd
import io

df = pd.read_csv('/content/sample_data/blackList.csv', encoding='windows-1252')
df.head()

,Entity,Name
0,Corporate,GANTABYA HOSPITAL AND RESEARCH CENTER
1,Individual,NABARAJ DEVKOTA
2,Individual,DADIKI SHERPA
3,Individual,KARAN CHAUDHARI THARU
4,Individual,KHIMRAJ RAI


In [ ]:
train_data = []

for _, row in df.iterrows():
  text = row['Name']
  label = row['Entity']
  label_dict = {'Individual':0,'Corporate':1}


  train_data.append((text, {'cats' : {'Individual' : 1 if label== 'Individual' else 0, 'Corporate': 1 if label == 'Corporate' else 0}}))





In [ ]:
print(train_data[:5])

[('GANTABYA HOSPITAL AND RESEARCH CENTER', {'cats': {'Individual': 0, 'Corporate': 1}}), ('NABARAJ DEVKOTA', {'cats': {'Individual': 1, 'Corporate': 0}}), ('DADIKI SHERPA', {'cats': {'Individual': 1, 'Corporate': 0}}), ('KARAN CHAUDHARI THARU', {'cats': {'Individual': 1, 'Corporate': 0}}), ('KHIMRAJ  RAI', {'cats': {'Individual': 1, 'Corporate': 0}})]


In [ ]:
nlp = spacy.blank("en")


text_classifier = nlp.add_pipe("textcat",last=True) #nlp.create_pipe("textcat",config={"architecture": "simple_cnn"})

text_classifier.add_label("Individual")
text_classifier.add_label("Corporate")

1

In [ ]:
import random

random.shuffle(train_data)

train_size = int(0.8 * len(train_data))
train_data_split = train_data[:train_size]
eval_data_split = train_data[train_size:]

In [ ]:
from spacy.util import minibatch
from spacy.training.example import Example

optimizers = nlp.begin_training()

for epoch in range(10):
  random.shuffle(train_data_split)
  losses = {}


  batches = minibatch(train_data_split,size=8)
  for batch in batches:
    texts, annotations = zip(*batch)

    examples = [Example.from_dict(nlp.make_doc(text),annotation) for text,annotation in zip(texts,annotations)]

    nlp.update(examples,losses=losses)

    print(f"Epoch {epoch+1} - Losses: {losses}")



Streaming output truncated to the last 5000 lines.
Epoch 10 - Losses: {'textcat': 3.34866323606665}
Epoch 10 - Losses: {'textcat': 3.348663236073929}
Epoch 10 - Losses: {'textcat': 3.3486632361137723}
Epoch 10 - Losses: {'textcat': 3.3486632365201583}
Epoch 10 - Losses: {'textcat': 3.3486632365255744}
Epoch 10 - Losses: {'textcat': 3.348663236533426}
Epoch 10 - Losses: {'textcat': 3.348663240948516}
Epoch 10 - Losses: {'textcat': 3.3486632604691606}
Epoch 10 - Losses: {'textcat': 3.3486632609263713}
Epoch 10 - Losses: {'textcat': 3.348663260927931}
Epoch 10 - Losses: {'textcat': 3.3486632609330353}
Epoch 10 - Losses: {'textcat': 3.348663261079621}
Epoch 10 - Losses: {'textcat': 3.348663261080841}
Epoch 10 - Losses: {'textcat': 3.348663261395025}
Epoch 10 - Losses: {'textcat': 3.348663261400989}
Epoch 10 - Losses: {'textcat': 3.3486632614062293}
Epoch 10 - Losses: {'textcat': 3.3486632614074465}
Epoch 10 - Losses: {'textcat': 3.348663261427668}
Epoch 10 - Losses: {'textcat': 3.348663272

In [ ]:
correct = 0
total = 0

for text,annotation in eval_data_split:
  doc = nlp.make_doc(text)
  true_label = max(annotation['cats'], key=annotation['cats'].get)
  if doc.cats:
     predicted_label = max(doc.cats,key=doc.cats.get)
  else:
    predicted_label = None;

  if true_label == predicted_label:
    correct += 1
    total += 1

    accuracy = correct / total

    print(f"Accuracy : {accuracy * 100: .2f}%")

In [ ]:
nlp.to_disk("corporate_individual_model");

In [ ]:
nlp = spacy.load("corporate_individual_model")

new_name = "Sunny"
doc = nlp(new_name)

print(f"Predicted label for '{new_name}' : {max(doc.cats,key=doc.cats.get)}")

Predicted label for 'Sunny Polytechniques' : Corporate
